In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tensorflow as tf
import random
import json
import pickle
import tflearn
from tflearn.layers.recurrent import lstm

with open('data.json') as f:
    data = json.load(f)
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.embedding(net, input_dim=len(training[0]), output_dim=128)  
net = lstm(net, 128, dropout=0.8)  
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")
model.save("model.h5")

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)


def chat():
    print("\033[1;31m Start talking with the bot!, Type quit when you want to quit")
    while True:
        inp = input("\033[1;31m You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:

            if tg['tag'] == tag:

                responses = tg['responses']

                response_list = nltk.sent_tokenize(str(responses[0]))
                colors = [31, 32, 33, 34, 35]

                for i in range(len(response_list)):
                    color_index = i % len(colors)
                    print(f'\033[1;{colors[color_index]}m {response_list[i]:<12s}\n')


chat()


Training Step: 35999  | total loss: 0.17980 | time: 2.359s
| Adam | epoch: 1000 | loss: 0.17980 - acc: 0.9539 -- iter: 280/284
Training Step: 36000  | total loss: 0.16360 | time: 2.422s
| Adam | epoch: 1000 | loss: 0.16360 - acc: 0.9586 -- iter: 284/284
--
INFO:tensorflow:F:\2024_april\Medical_Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:F:\2024_april\Medical_Chatbot\model.h5 is not in all_model_checkpoint_paths. Manually adding it.
 Start talking with the bot!, Type quit when you want to quit
 You: hi
 Hello!      

 How can I assist you today?

 You: quit


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import json
import tflearn
import numpy as np
import tensorflow as tf

# Download nltk data
nltk.download('punkt')

# Initialize stemmer
stemmer = LancasterStemmer()

# Load intents data
with open('data.json') as file:
    data = json.load(file)

# Load words, labels, training data, and output from the data
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in '?']
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

# Reset tensorflow graph
tf.compat.v1.reset_default_graph()

# Build neural network
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.embedding(net, input_dim=len(training[0]), output_dim=128)  
net = tflearn.lstm(net, 128, dropout=0.8)  
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

# Initialize model
model = tflearn.DNN(net)

# Load the trained model
model.load("model.h5")


def chat():
    print("\033[1;31m Start talking with the bot!, Type quit when you want to quit")
    while True:
        inp = input("\033[1;31m You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:

            if tg['tag'] == tag:

                responses = tg['responses']

                response_list = nltk.sent_tokenize(str(responses[0]))
                colors = [31, 32, 33, 34, 35]

                for i in range(len(response_list)):
                    color_index = i % len(colors)
                    print(f'\033[1;{colors[color_index]}m {response_list[i]:<12s}\n')


chat()
